In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
from typing import Dict, List

import boto3
import requests


S3_BUCKET = "cubix-chicago-taxi-bb"
TAXI_API_URL = "https://data.cityofchicago.org/resource/ajtu-isnz.json"
WEATHER_API_URL = "https://archive-api.open-meteo.com/v1/era5"

s3_client = boto3.client("s3")


def get_data_from_api(url: str, params: Dict = None) -> List[Dict]:
    """
    Retrieves data from the given api with parameters.
    
    :param url:     The URL to retrieve data from.
    :param params:  Optionally send parameters with the request.
    :return:        A list of dictionaries contatining the data.
    """
    print(f"Fetching data from {url}")
    try:
        response = requests.get(url, params=params)
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return None


def get_taxi_data(date_str: str) -> List[Dict]:
    """
    Retrieves taxi data for the given date.

    :param date_str:    The date in "YYYY-MM-DD" format.
    :return:            A list of dictionaries contatining the taxi_data as a JSON.
    """
    query = f"?$where=trip_start_timestamp >= '{date_str}T00:00:00' AND trip_start_timestamp <= '{date_str}T23:59:59'&$limit=30000"
    return get_data_from_api(TAXI_API_URL + query)


def get_weather_data(date_str: str) -> Dict:
    """
    Retrieve weather data from the Open Meteo API for a specific date and location.

    :param formatted_datetime:  The date in "YYYY-MM-DD" format.
    :return:                    A dictionary containing weather data, including temperature at 2 meters, wind speed at 10 meters,
                                rain, and precipitation for the specified date and location.
    """
    params = {
        "latitude": 41.85,
        "longitude": -87.65,
        "start_date": date_str,
        "end_date": date_str,
        "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
    }
    return get_data_from_api(WEATHER_API_URL, params)


def upload_to_s3(data: Dict, folder: str, filename: str) -> None:
    """Upload data to an S3 bucket.
    
    :param data:            A list of dictionaries containing the data to be uploaded, either taxi or weather data.
    :param folder_name:     The name of the folder within the S3 bucket where the data will be stored.
    :param filename:        The name of the file to be created within the specified folder.
    :raise ValueError:      If 'data' is None or empty ({} or []).
    :raise Boto3Error:      If there is an error uploading to S3.
    """
    if not data:
        raise ValueError(f"Data for {filename} is empty! Stopping execution.")

    s3_key = f"raw_data/to_processed/{folder}/{filename}"
    
    try:
        s3_client.put_object(
            Bucket=S3_BUCKET,
            Key=s3_key,
            Body=json.dumps(data)
        )
        print(f"Successfully uploaded {filename} to S3: {s3_key}")
    except boto3.exceptions.Boto3Error as e:
        print(f"Failed to upload {filename} to S3: {e}")


def lambda_handler(event, context):
    """
    Getting the raw taxi and weather data from their respective APIs, and extract to S3.

    Steps:
        1. Create the date which is today minus 2 months, for continuity.
        2. Get the taxi raw data.
        3. Get the weather raw data.
        4. Upload them to S3. 
    """
    date_str = (datetime.now() - relativedelta(months=2)).strftime("%Y-%m-%d")

    taxi_data = get_taxi_data(date_str)
    weather_data = get_weather_data(date_str)

    upload_to_s3(taxi_data, "taxi_data", f"taxi_raw_{date_str}.json")
    upload_to_s3(weather_data, "weather_data", f"weather_raw_{date_str}.json")
